# Prepare DoS Goldeneye Attack Dataset

## Overview:

This notebook will focus on creating a DoS Goldeneye attack dataset based on a small sample of data collected by performing real DoS HTTP GET Flood attacks in a controlled environment.<br>
The dataset that this notebook creates closely represents real-world data and was used to train our SVM model.<br>  
It is worth noteing that the sample dataset we collected does not contain any missing values or any outliers due to the fact we tested each part of the collection process and verified that it is correct.<br>
In this notebook we have generated an attack dataset with 9,000 flows of the DoS HTTP GET Flood attack based on the samples we collected when running a DoS HTTP GET Flood attacks in various configurations using the well known DoS Goldeneye tool when the victim web server was online.<br> 
The victim web server was a regular Flask web server.

## Imports & Global Variables:

In [2]:
import pandas as pd
import numpy as np
import random

NUM_OF_ROWS = 9000
ATTACK_NAME = 'DoS'

In [3]:
# the following command will make it so that when we print the dataframe we will see all the columns
pd.set_option('display.max_columns', None)

---

## Load the sample dataset:

In [4]:
# import the attack sample dataset
dos_samples = pd.read_csv('dos_goldeneye_samples.csv') 
print(f'Dataset Shape: {dos_samples.shape}')
dos_samples

Dataset Shape: (21, 26)


,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,124.775482,66,845,145.078895,21047.885732,584231,811,90.775482,32,145.078895,0,0.0,0,0.0,55.619484,0.0,117.268366,2539,3897,0,5.743676,1120.536707,0.023122,0.000893,0.001892
1,1,121.000527,66,849,140.240047,19667.270800,825026,815,87.000527,32,140.240047,0,0.0,0,0.0,52.001476,0.0,97.084726,3555,5928,0,10.782026,879.519319,0.029030,0.001137,0.002449
2,1,122.496462,66,860,141.830236,20115.815856,850451,826,88.496462,32,141.830236,0,0.0,0,0.0,53.694381,0.0,96.074446,3366,6244,0,11.796613,814.640611,0.078117,0.001228,0.002738
3,1,122.480506,66,829,142.044040,20176.509262,839680,795,88.480506,32,142.044040,0,0.0,0,0.0,53.649737,0.0,95.083230,3358,6132,0,12.465338,761.311086,0.051932,0.001314,0.002873
4,1,123.573296,66,831,143.762818,20667.747896,863397,797,89.573296,32,143.762818,0,0.0,0,0.0,54.859322,0.0,96.221665,3270,6369,0,12.742510,756.444383,0.061831,0.001322,0.002859
5,1,123.609547,66,881,143.952884,20722.432703,856040,847,89.609547,32,143.952884,0,0.0,0,0.0,54.859416,0.0,97.199955,3284,6269,0,14.330901,666.601486,0.063931,0.001500,0.003420
6,1,104.405674,54,833,120.419451,14500.844118,672515,799,70.405674,20,120.419451,0,0.0,0,0.0,36.545959,0.0,74.607832,2519,6834,3074,11.847213,806.265573,3.898364,0.001240,0.040068
7,1,109.580024,54,792,126.620150,16032.662369,690877,758,75.580024,20,126.620150,0,0.0,0,0.0,41.594246,0.0,82.345292,2911,5802,1722,14.243538,641.764701,5.236684,0.001558,0.054854
8,1,122.578599,66,813,141.961046,20152.938582,802965,779,88.578599,32,141.961046,0,0.0,0,0.0,53.393602,0.0,105.932058,3609,5456,0,8.945775,1013.327517,0.137966,0.000987,0.003047
9,1,106.246133,54,911,122.568377,15023.007152,541846,877,72.246133,20,122.568377,0,0.0,0,0.0,38.640800,0.0,81.005531,1871,5385,1990,7.606195,986.038358,0.115753,0.001014,0.003427


### Find the columns that we need to synthesis data for:

In [ ]:
columns_to_gather = dos_samples.replace(0, np.nan) #replace all 0 values with null
columns_to_gather = columns_to_gather.dropna(how = 'all', axis = 1).columns.tolist() #remove all columns where there are null values
columns_to_gather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Packet Length Std',
 'Packet Length Variance',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Fwd Packet Length Std',
 'Fwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'ACK Flag Count',
 'RST Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

### Find an approximate minimum and maximum values of each column:

The 'RST Flag Count' column sometimes has values and other times has the value 0, there for for this specific column<br>
we decided to calculate the minimum/maximum values based on actual numbers without the 0, and later to add rows where the<br>
value will be exactly 0 in order to match the collect sample dataset.

In [ ]:
# find the minimum and maximum values for each column, scale the range (reduce min by 15% and increase max by 15%), and store the results in a dictionary.
RST_FlagCount = dos_samples[dos_samples['RST Flag Count'] != 0]['RST Flag Count'] 

min_max_dict = {col: (dos_samples[col].min() * 0.85, dos_samples[col].max() * 1.15) for col in columns_to_gather}
min_max_dict['Number of Ports'] = (1, 1)
min_max_dict['RST Flag Count'] = (np.min(RST_FlagCount), np.max(RST_FlagCount))

# print the min max dictionary
for col, (min_val, max_val) in min_max_dict.items():
    print(f'{col:<30} | Min: {min_val:.2f} | Max: {max_val:.2f}')

Number of Ports                | Min: 1.00 | Max: 1.00
Average Packet Length          | Min: 88.74 | Max: 146.11
Packet Length Min              | Min: 45.90 | Max: 75.90
Packet Length Max              | Min: 673.20 | Max: 1047.65
Packet Length Std              | Min: 102.36 | Max: 169.56
Packet Length Variance         | Min: 12325.72 | Max: 25000.66
Total Length of Fwd Packet     | Min: 378120.80 | Max: 1004422.65
Fwd Packet Length Max          | Min: 644.30 | Max: 1008.55
Fwd Packet Length Mean         | Min: 59.84 | Max: 107.01
Fwd Packet Length Min          | Min: 17.00 | Max: 36.80
Fwd Packet Length Std          | Min: 102.36 | Max: 169.56
Fwd Segment Size Avg           | Min: 31.06 | Max: 67.51
Subflow Fwd Bytes              | Min: 63.42 | Max: 134.86
SYN Flag Count                 | Min: 1489.20 | Max: 4242.35
ACK Flag Count                 | Min: 3211.30 | Max: 7859.10
RST Flag Count                 | Min: 95.00 | Max: 3074.00
Flow Duration                  | Min: 4.88 | Max: 39

### Create the base attack dataset (full of zeros):

In [7]:
# creating an empty dataframe before adding values to it
dos_dataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dos_samples.columns))), columns = dos_samples.columns)
dos_dataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Find the columns with constant zero values based on samples:

In [ ]:
# adding zeros to all columns that should not have any values
zero_columns = [col for col in dos_samples.columns if col not in columns_to_gather]
for col in zero_columns:
    dos_dataset[col] = int(0)
zero_columns

['Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg']

In [9]:
dos_dataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

## Filling in values based on collected samples:

## Calculate and fill values into columns that have a certain correlation between them:

A correlation between two or more columns is common in our dataset since most features are inherently related. All of them are derived from network packet traffic.<br>
For example, as the **flow duration** increases, the **packets per second** is likely to decrease. This occurs because each flow has an upper limit on duration, after which data collection stops and a new flow begins.<br>  
Similarly, the **Inter-Arrival Time (IAT)** of packets within a flow is influenced by the flow duration. Given these dependencies, <br>
the attack dataset should generate data for these columns collectively, ensuring that their inherent correlations are maintained.

### Correlation between 'SYN Flag Count' and 'ACK Flag Count':

In [ ]:
# finding the correlation between the 'SYN Flag Count' column to the rest of the columns in order to create new data
first_correlation = ['SYN Flag Count', 'ACK Flag Count']
independent_col = dos_samples[first_correlation[0]].values.reshape(-1, 1) #column 'SYN Flag Count'
dependent_cols = dos_samples[first_correlation[1]].values #the rest of the columns that are not zeros

# using least squares regression to find scaling factors that best approximate the relationship between 'SYN Flag Count' and 'ACK Flag Count'
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name,factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('ACK Flag Count', np.float64(1.8289174466546883))


After finding the scaling factors we can apply some randomness when generating values for the attack dataset in order to generate better data (without many duplications).<br>
We add randomness by creating a modified scaling factor, which introduces controlled variations in the generated values.<br>
This is done by selecting a small random delta (between 10% and 20% of the factor) and adding or subtracting it from the original scaling factor.<br>
As a result, the generated data maintains realistic correlations while avoiding exact duplicates.

In [11]:
dos_dataset['SYN Flag Count'] = np.random.randint(min_max_dict['SYN Flag Count'][0], min_max_dict['SYN Flag Count'][1], NUM_OF_ROWS)

# generate new data by scaling the original correlated column value using the updated factor.
for index, row in dos_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all generated scaling factors
        delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
        updated_factor = factor + random.choice([-1, 1]) * delta
        dos_dataset.loc[index, col] = int(row['SYN Flag Count'] * updated_factor)

In [12]:
dos_dataset.head(10)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3063,6355.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3905,5816.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2353,4995.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1604,3323.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2266,3358.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3456,5240.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3361,7183.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2638,3973.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3175,6470.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2533,4088.0,0.0,0.0,0.0,0.0,0.0,0.0


As we mentioned above, the 'RST Flag Count' column in this attack sample contains two different values, sometimes its exactly zero, and other times its a number in a specific range.<br> 
There for we insert data into this column such that half of the vector will be zeros and the other half will be values in the range of usual values (between minimal value that is not zero and the maximum value).

In [13]:
# adding the RST flag such that half of the rows will have 0 and the other half will have a random value from the known range
midpoint = len(dos_dataset) // 2 #calculate the midpoint of the DataFrame

# Generate random integers for the second half
random_values = np.random.randint(min_max_dict['RST Flag Count'][0], min_max_dict['RST Flag Count'][1] + 1, size=len(dos_dataset) - midpoint)

# Add the new column with 0s for the first half and random integers for the second half
new_column_values = [0] * midpoint + list(random_values)
dos_dataset['RST Flag Count'] = new_column_values

In [14]:
dos_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3063,6355.0,0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3905,5816.0,0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2353,4995.0,0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1604,3323.0,0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2266,3358.0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2265,3569.0,766,0.0,0.0,0.0,0.0,0.0
8996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3901,6027.0,2170,0.0,0.0,0.0,0.0,0.0
8997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3181,6464.0,2663,0.0,0.0,0.0,0.0,0.0
8998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3328,6699.0,1333,0.0,0.0,0.0,0.0,0.0


### Correlation between 'Flow Duration' and all of the following: 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std':

In [ ]:
# Generate random values for the 'Flow Duration' column
rand_values = np.random.uniform(min_max_dict['Flow Duration'][0]*0.9, min_max_dict['Flow Duration'][1]*1.05, size = NUM_OF_ROWS)

# Assign the random values
dos_dataset['Flow Duration'] = rand_values

In [ ]:
# finding the correlation between the 'Flow Duration' column to the rest of the columns in order to create new data
second_correlation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = dos_samples[second_correlation[0]].values.reshape(-1, 1) #column 'Flow Duration'
dependent_cols = dos_samples[second_correlation[1:]].values 

# using least squares regression to find scaling factors that best approximate the relationship between 'Flow Duration' and the rest of the columns in second_correlation
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name,factor) for name, factor in zip(second_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packets Per Second', np.float64(30.145241232971678))
('IAT Max', np.float64(0.3696369152684689))
('IAT Mean', np.float64(0.00011623768083943193))
('IAT Std', np.float64(0.0041811844462341175))


In [17]:
# calculate the average correlation between flow duration and packets per second by multiplying their corresponding values from both columns and then calculate the average.
duration_to_packets_corr = [x * y for x, y in zip(dos_samples['Flow Duration'].values, dos_samples['Packets Per Second'].values)]
duration_to_packets_corr = np.mean(duration_to_packets_corr)
duration_to_packets_corr

np.float64(8732.047619047618)

And again here after finding the scaling factors we add some randomness and generate the data

In [18]:
# calculate a random small delta of the factor for some randomness
for index, row in dos_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Flow Duration'
        if col == 'Packets Per Second':
            delta = random.uniform(duration_to_packets_corr * 0.1, duration_to_packets_corr * 0.2) 
            updated_factor = duration_to_packets_corr + random.choice([-1, 1]) * delta
            dos_dataset.loc[index, col] = updated_factor / row['Flow Duration']
        else:
            if col == 'IAT Std' or col == 'IAT Max':
                delta = random.uniform(factor * 0.7, factor * 0.99)
                updated_factor = factor + random.choices([-1, 1], weights=[2, 1], k=1)[0] * delta  
            else:
                delta = random.uniform(factor * 0.1, factor * 0.2) 
                updated_factor = factor + random.choice([-1, 1]) * delta
            dos_dataset.loc[index, col] = row['Flow Duration'] * updated_factor

### Correlation between 'Average Packet Length' and all of the following:<br>'Packet Length Std', 'Packet Length Variance', 'Total Length of Fwd Packet', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Fwd Segment Size Avg', 'Subflow Fwd Bytes':

In [ ]:
# finding the correlation between the 'Average Packet Length' column to the rest of the columns in order to create new data
first_correlation = ['Average Packet Length', 'Packet Length Std', 'Packet Length Variance', 'Total Length of Fwd Packet', 
                    'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Fwd Segment Size Avg', 'Subflow Fwd Bytes']
independent_col = dos_samples[first_correlation[0]].values.reshape(-1, 1) #column 'Average Packet Length'
dependent_cols = dos_samples[first_correlation[1:]].values 

# using least squares regression to find scaling factors that best approximate the relationship between Average Packet Length' and the rest of the columns in second_correlation
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name,factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packet Length Std', np.float64(1.1607160816175794))
('Packet Length Variance', np.float64(160.25967924010962))
('Total Length of Fwd Packet', np.float64(6271.216232539372))
('Fwd Packet Length Mean', np.float64(0.7132122210543036))
('Fwd Packet Length Std', np.float64(1.1607160816175794))
('Fwd Segment Size Avg', np.float64(0.4210840873569066))
('Subflow Fwd Bytes', np.float64(0.8054518323891989))


And again here after finding the scaling factors we add some randomness and generate the data

In [ ]:
dos_dataset['Average Packet Length'] = np.random.uniform(min_max_dict['Average Packet Length'][0]*0.85, min_max_dict['Average Packet Length'][1]*1.15, NUM_OF_ROWS)

for index, row in dos_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Average Packet Length'
        delta = random.uniform(factor * 0.1, factor * 0.2) 
        updated_factor = factor + random.choice([-1, 1]) * delta
        dos_dataset.loc[index, col] = row['Average Packet Length'] * updated_factor

### Then we insert data into columns that are independant of each other:

In [ ]:
independant_columns = ['Packet Length Min', 'Packet Length Max', 'Fwd Packet Length Max', 'Fwd Packet Length Min']

for col in independant_columns:
    dos_dataset[col] = (np.random.uniform(min_max_dict[col][0]*0.85, min_max_dict[col][1]*1.15, NUM_OF_ROWS)).astype(int)

dos_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,141.778729,67,731,184.978750,26327.138038,7.925931e+05,773,86.936344,23,193.696789,0,0,0,0,68.249420,0,92.647498,3063,6355.0,0,8.282244,1172.252487,0.253250,0.000828,0.006945
1,0.0,97.813387,54,889,93.434574,17548.740804,5.016908e+05,779,81.636842,16,133.218699,0,0,0,0,46.879070,0,65.940726,3905,5816.0,0,15.653459,454.481331,0.796145,0.002169,0.112880
2,0.0,139.738207,76,1190,181.110497,19277.036051,7.294678e+05,581,83.827870,29,132.259305,0,0,0,0,66.680296,0,124.277839,2353,4995.0,0,33.644347,304.808400,0.702499,0.004469,0.019248
3,0.0,162.571792,57,808,220.065169,22431.764611,1.189967e+06,633,102.487927,37,166.707911,0,0,0,0,81.333712,0,149.701200,1604,3323.0,0,34.347537,227.391250,2.231734,0.004600,0.004996
4,0.0,144.902165,73,936,195.769254,27211.165634,7.455935e+05,721,115.194617,41,147.531733,0,0,0,0,67.679118,0,101.196307,2266,3358.0,0,8.827874,795.489908,0.753134,0.001130,0.000765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,0.0,98.536041,87,747,94.297076,13055.448222,7.067221e+05,1033,60.322695,29,132.240095,0,0,0,0,35.507742,0,69.794410,2265,3569.0,766,16.660866,435.087720,0.897409,0.002227,0.132456
8996,0.0,142.595471,49,741,197.018158,18892.232419,1.006184e+06,855,88.288779,32,132.607153,0,0,0,0,69.527965,0,136.745232,3901,6027.0,2170,19.008497,403.622036,13.442931,0.002534,0.135983
8997,0.0,104.649778,55,730,144.978477,18684.060838,5.532446e+05,1009,59.931815,26,137.490901,0,0,0,0,51.167905,0,72.179355,3181,6464.0,2663,30.498974,250.955907,19.222976,0.004007,0.014423
8998,0.0,166.055474,46,957,172.320760,30171.143473,9.169478e+05,815,95.648953,17,227.978536,0,0,0,0,62.864908,0,157.936450,3328,6699.0,1333,18.285979,402.820890,11.963856,0.001837,0.147071


---

## Adding the Label and Number of Ports column:

In [22]:
# adding number of ports and a label to the dataset
dos_dataset['Number of Ports'] = np.full(shape = NUM_OF_ROWS, fill_value = 1, dtype = int)
dos_dataset['Label'] = ATTACK_NAME

## Validate that the generated data looks valid by comparing the samples with the generated dataset:

In [23]:
dos_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std,Label
0,1,141.778729,67,731,184.978750,26327.138038,7.925931e+05,773,86.936344,23,193.696789,0,0,0,0,68.249420,0,92.647498,3063,6355.0,0,8.282244,1172.252487,0.253250,0.000828,0.006945,DoS
1,1,97.813387,54,889,93.434574,17548.740804,5.016908e+05,779,81.636842,16,133.218699,0,0,0,0,46.879070,0,65.940726,3905,5816.0,0,15.653459,454.481331,0.796145,0.002169,0.112880,DoS
2,1,139.738207,76,1190,181.110497,19277.036051,7.294678e+05,581,83.827870,29,132.259305,0,0,0,0,66.680296,0,124.277839,2353,4995.0,0,33.644347,304.808400,0.702499,0.004469,0.019248,DoS
3,1,162.571792,57,808,220.065169,22431.764611,1.189967e+06,633,102.487927,37,166.707911,0,0,0,0,81.333712,0,149.701200,1604,3323.0,0,34.347537,227.391250,2.231734,0.004600,0.004996,DoS
4,1,144.902165,73,936,195.769254,27211.165634,7.455935e+05,721,115.194617,41,147.531733,0,0,0,0,67.679118,0,101.196307,2266,3358.0,0,8.827874,795.489908,0.753134,0.001130,0.000765,DoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,1,98.536041,87,747,94.297076,13055.448222,7.067221e+05,1033,60.322695,29,132.240095,0,0,0,0,35.507742,0,69.794410,2265,3569.0,766,16.660866,435.087720,0.897409,0.002227,0.132456,DoS
8996,1,142.595471,49,741,197.018158,18892.232419,1.006184e+06,855,88.288779,32,132.607153,0,0,0,0,69.527965,0,136.745232,3901,6027.0,2170,19.008497,403.622036,13.442931,0.002534,0.135983,DoS
8997,1,104.649778,55,730,144.978477,18684.060838,5.532446e+05,1009,59.931815,26,137.490901,0,0,0,0,51.167905,0,72.179355,3181,6464.0,2663,30.498974,250.955907,19.222976,0.004007,0.014423,DoS
8998,1,166.055474,46,957,172.320760,30171.143473,9.169478e+05,815,95.648953,17,227.978536,0,0,0,0,62.864908,0,157.936450,3328,6699.0,1333,18.285979,402.820890,11.963856,0.001837,0.147071,DoS


In [24]:
dos_samples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,21.0,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.0,21.0,21.0,21.0,21.000000,21.0,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
mean,1.0,118.162351,62.000000,835.333333,137.140846,18874.235571,737972.047619,801.333333,84.162351,28.000000,137.140846,0.0,0.0,0.0,0.0,49.548964,0.0,95.038656,3010.761905,5627.904762,562.238095,15.224132,700.470398,4.264842,0.001767,0.049179
std,0.0,6.975820,5.796551,29.169048,8.363914,2241.856330,128170.354837,29.169048,6.975820,5.796551,8.363914,0.0,0.0,0.0,0.0,6.637618,0.0,9.910132,551.209389,852.917400,897.206103,7.796555,276.305968,6.595806,0.000968,0.074095
min,1.0,104.405674,54.000000,792.000000,120.419451,14500.844118,444848.000000,758.000000,70.405674,20.000000,120.419451,0.0,0.0,0.0,0.0,36.545959,0.0,74.607832,1752.000000,3778.000000,0.000000,5.743676,246.263767,0.023122,0.000893,0.001892
25%,1.0,109.976270,54.000000,818.000000,127.842838,16343.791319,640328.000000,784.000000,75.976270,20.000000,127.842838,0.0,0.0,0.0,0.0,41.808258,0.0,88.539871,2656.000000,5385.000000,0.000000,9.452635,457.067517,0.063931,0.001061,0.002862
50%,1.0,121.000527,66.000000,831.000000,140.240047,19667.270800,773385.000000,797.000000,87.000527,32.000000,140.240047,0.0,0.0,0.0,0.0,52.030510,0.0,96.074446,3234.000000,5802.000000,0.000000,12.632054,756.444383,0.137966,0.001322,0.003427
75%,1.0,123.044975,66.000000,849.000000,142.734482,20373.132473,846118.000000,815.000000,89.044975,32.000000,142.734482,0.0,0.0,0.0,0.0,53.888530,0.0,101.706229,3366.000000,6244.000000,1035.000000,19.535407,943.017495,5.236684,0.002188,0.054854
max,1.0,127.054656,66.000000,911.000000,147.443905,21739.705195,873411.000000,877.000000,93.054656,32.000000,147.443905,0.0,0.0,0.0,0.0,58.701364,0.0,117.268366,3689.000000,6834.000000,3074.000000,34.094212,1120.536707,20.601452,0.004061,0.230153


In [25]:
dos_dataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,9000.0,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9.000000e+03,9000.000000,9000.000000,9000.000000,9000.000000,9000.0,9000.0,9000.0,9000.0,9000.000000,9000.0,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000
mean,1.0,121.867609,62.562333,888.719667,141.687668,19567.032822,7.655892e+05,854.943000,86.663550,27.844444,141.566859,0.0,0.0,0.0,0.0,51.324956,0.0,97.888623,2876.956889,5258.727222,799.382444,22.566661,535.570932,6.138094,0.002617,0.068169
std,0.0,26.788035,13.944587,182.954749,38.120351,5290.078952,2.065302e+05,176.176359,23.328489,8.037432,38.194435,0.0,0.0,0.0,0.0,13.902213,0.0,26.394881,794.908877,1681.691531,1007.876618,10.579885,389.003766,8.019350,0.001298,0.090142
min,1.0,75.433134,39.000000,572.000000,70.514975,9736.001756,3.814924e+05,547.000000,43.382169,14.000000,70.784620,0.0,0.0,0.0,0.0,25.445613,0.0,48.834259,1489.000000,2213.000000,0.000000,4.394145,170.213855,0.023059,0.000412,0.000226
25%,1.0,98.690628,50.000000,733.000000,112.237028,15488.107214,6.050378e+05,704.000000,68.895674,21.000000,111.885073,0.0,0.0,0.0,0.0,40.502526,0.0,77.487534,2185.750000,3901.750000,0.000000,13.487587,273.206175,0.755214,0.001521,0.008306
50%,1.0,122.283088,63.000000,891.000000,138.108641,19115.081255,7.468647e+05,857.000000,84.368285,28.000000,138.353261,0.0,0.0,0.0,0.0,50.076043,0.0,95.194715,2887.000000,5145.500000,47.500000,22.344869,391.510644,1.897628,0.002550,0.021394
75%,1.0,144.762377,74.000000,1045.000000,166.651056,22961.053556,9.011376e+05,1007.000000,101.688202,35.000000,166.838172,0.0,0.0,0.0,0.0,60.398706,0.0,114.942760,3566.000000,6371.000000,1615.000000,31.679681,647.146489,9.646288,0.003594,0.102520
max,1.0,167.992389,87.000000,1204.000000,233.091393,32183.330701,1.260956e+06,1159.000000,142.829411,42.000000,233.746660,0.0,0.0,0.0,0.0,84.728364,0.0,161.830271,4241.000000,9284.000000,3074.000000,41.166389,2375.018207,29.785379,0.005708,0.339249


---

## At the end we save the dataset as a CSV file

In [26]:
print(f'Attack Dataset Shape: {dos_dataset.shape}')

Attack Dataset Shape: (9000, 27)


In [ ]:
# save the dataset
dos_dataset.to_csv('dos_goldeneye_dataset.csv', index=False)

---